## License Plate Detection using Fuzzy Join 

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

### Connect to EvaDB

In [27]:
%pip install "evadb[vision,notebook]"
%pip install thefuzz
%pip install "protobuf==3.20.1"
import evadb
cursor = evadb.connect().cursor()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` befor

### Loading the images to EvaDB for analysis

In [28]:
# Download images
!wget -nc "https://www.dropbox.com/s/770stddqfl0psog/license.zip"
!unzip -n license.zip

cursor.query('DROP TABLE IF EXISTS MyImages;').df()

cursor.load("license/Car*.png", "MyImages", format="image").df()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
File 'license.zip' already there; not retrieving.

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Archive:  license.zip


,0
0,Number of loaded IMAGE: 7


### License Plate Recognition

In [29]:
cursor.query("""DROP UDF IF EXISTS OCRExtractor;""").df()

cursor.query("""DROP UDF IF EXISTS FuzzDistance;""").df()

cursor.create_udf("OCRExtractor", True, '../evadb/udfs/ocr_extractor.py').df()
cursor.create_udf("FuzzDistance", True, '../evadb/udfs/fuzzy_join.py').df()

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


,0
0,UDF FuzzDistance successfully added to the dat...


In [30]:
cursor.query(
    "CREATE TABLE IF NOT EXISTS LicensePlateCSV(id INTEGER UNIQUE, label TEXT(30));"
).df()

06-24-2023 16:24:47 WARNING[executor_utils:executor_utils.py:handle_if_not_exists:0083] Table: LicensePlateCSV already exists


""


In [31]:
cursor.load("/home/jarulraj3/eva/tutorials/data.csv", "LicensePlateCSV",format="csv").df()

,CSV,Number of loaded frames
0,/home/jarulraj3/eva/tutorials/data.csv,5


In [32]:
import logging
logging.getLogger('sqlalchemy.engine').setLevel(logging.INFO)
query = cursor.table("MyImages")
query = query.select("OCRExtractor(data)")
print(query.df())

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


06-24-2023 16:24:58 ERROR [base_model:base_model.py:_commit:0097] Database commit failed : (sqlite3.OperationalError) database is locked
(Background on this error at: https://sqlalche.me/e/14/e3q8)
06-24-2023 16:24:58 ERROR [base_model:base_model.py:save:0058] Database save failed : (sqlite3.OperationalError) database is locked
(Background on this error at: https://sqlalche.me/e/14/e3q8)
06-24-2023 16:24:58 ERROR [plan_executor:plan_executor.py:execute_plan:0172] Error while upserting entry to UdfCostCatalog: Error while inserting entry to UdfCostCatalog: (sqlite3.OperationalError) database is locked
(Background on this error at: https://sqlalche.me/e/14/e3q8)
Traceback (most recent call last):
  File "/nethome/jarulraj3/eva/test_evadb/lib/python3.10/site-packages/sqlalchemy/engine/base.py", line 1094, in _commit_impl
    self.engine.dialect.do_commit(self.connection)
  File "/nethome/jarulraj3/eva/test_evadb/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 686, in do_c

ExecutorError: Error while upserting entry to UdfCostCatalog: Error while inserting entry to UdfCostCatalog: (sqlite3.OperationalError) database is locked
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
query = cursor.table("LicensePlateCSV")
query = query.select("*")

query.df()

### Run Fuzzy Join to match Detected License Plate against Local License Plate Database (csv)

In [ ]:
cursor.query("""
   SELECT * FROM MyImages 
       JOIN LATERAL OCRExtractor(data) AS T(a) 
       JOIN LicensePlateCSV B 
       ON FuzzDistance(T.a, B.label) > 50
       WHERE id < 3;
       """).df()